## 4.6 랜덤 포레스트의 특성 중요도 사용

앙상블에 참여한 모든 결정 트리에서 계산한 평균적인 불순도 감소로 특성 중요도를 측정 가능  
사이킷런의 랜덤 포레스트 구현은 특성 중요도 값을 수집해 놓음  
`RandomForestClassifier` 클래스에서 `feature_importances_` 속성으로 확인  

In [ ]:
from sklearn.ensemble import RandomForestClassifier
feat_labels = df_wine.columns[1:]
forest = RandomForestClassifier(n_estimators=500, random_state=1)
forest.fit(X_train, y_train)
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f+1, 30, feat_labels[indices[f]],
                            importances[indices[f]]))
    
plt.title('Feature importance')
plt.bar(range(X_train.shape[1]),
        importances[indices],
        align='center')

plt.xticks(range(X_train.shape[1]),
           feat_labels[indices], rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()
plt.show()

`SelectFromModel`: 사용자가 지정한 임계 값을 기반으로 특성을 선택  
* `coef_`
* `feature_importances_`
* `importance_getter` 매개변수
* `threshold` 매개변수 (default: mean)

In [ ]:
from sklearn.feature_selection import SelectFromModel

sfm = SelectFromModel(forest, threshold=0.1, prefit=True)
X_selected = sfm.transform(X_train)
print('이 임계 조건을 만족하는 샘플의 수: ', X_selected.shape[1])

In [ ]:
for f in range(X_selected.shape[1]):
    print("%2d) %-*s %f" % (f+1, 30, 
                            feat_labels[indices[f]],
                            importances[indices[f]]))

*REFERENCE)*  
RFE (재귀적 특성 제거 방법): 처음에 모든 특성을 사용하여 모델링, 특성 중요도가 가장 낮은 특성을 제거  
그 다음 제외된 특성을 빼고 나머지 특성으로 새로운 모델을 만듦  
미리 정의한 특성 개수가 남을 때까지 계속  

* `n_features_to_select` 매개변수: 선택할 특성의 개수, 비율을 지정
* `step` 매개변수: 각 반복에서 제거할 특성의 개수를 지정
* `coef_`, `feature_importances_` 속성을 기준으로 특성을 제거
* `importance_getter` 매개변수: 사용할 속성을 지정할 수 있음
* `ranking_` 속성: 선택한 특성의 우선순위
* `support_` 속성: 선택된 특성
* `estimator_` 속성: 훈련된 기반 모델 (랜덤 포레스트)

In [ ]:
from sklearn.feature_selection import RFE
rfe = RFE(forest, n_features_to_select=5)
rfe.fit(X_train, y_train)

In [ ]:
rfe.ranking_
f_mask = rfe.support_

In [ ]:
importances = rfe.estimator_.feature_importances_
indices = np.argsort(importances)[::-1]
for i in indices:
    print("%2d) %-*s %f" % (f+1, 30, 
                            feat_labels[f_mask][i],
                            importances[i]))